# Deploy Media Graph

In this section, we will deploy media graphs onto our IoT Edge device to trigger our modules to perform certain actions.

## Deploy / Manage Media Graph on LVA Edge Module
At this stage of the sample, we should have all required modules deployed to the Edge device, up and running.  

Now we will deploy media graphs onto the "lvaEdge" module. Here, "lvaEdge" is the name that we assigned to the LVA module inside the module deployment template json file from the [previous sample](deploy_iotedge_modules.ipynb). Media graph is also a Json file that defines a media flow pipeline from media sources to the target sinks and any analytics processes in-between. The media graph file consists of following modules:  

```
{
    "@apiVersion": "1.0",

    "name": "SampleMediaGraphOrganization",

    "properties":{

        "parameters": ...

        "sources": ...

        "processes": ...

        "sinks": ...
    }
}
```

In the media graph Json file above, the "parameters" section defines default values of all required parameters to run the media graph. In this section, we will define the addresses of the media sources, their access credentials, and the customized parameters (to override the default values) related to the process modules, such as fps rate and sensitivity of motion detection. Each media graph may have a different media flow, i.e., one may have a motion detection processor while another may not. You may modify this media graph according to your needs.  

### Sample Media Graph
We will use a sample media graph file as defined in setup_specific_environment.ipynb under the variable `topologyFile`. For details about the functionality of this sample media graph, you can check the details in the readme file in the same folder as `topologyFile`. You can also find more media graph samples under the MediaGraph/topologies/ folder.   

For more, refer to [details about LVA media graphs](https://docs.microsoft.com/en-us/azure/media-services/live-video-analytics-edge/media-graph-concept). 

### Setting Parameters in Media Graph
There are two ways to set parameters in the media graph:  

1. Update the default values of the parameters in "template.json", which is **not** the preferred method. 
2. Note the parameter names and data types in "template.json", and set their values in the deployment command that we will show in the next section.

### Deploy the Media Graph with Custom Parameters
Below, we will explain the following operations:

1. Setting (deploying) Media Graph Topology on the IoT Edge Device's lvaEdge module
2. Creating a Media Topology Instance from the one set in step 1
3. Activate the Topology Instance that set in step 2

## Import A Helper Python Class to Deploy Media Graphs
The following code snippet will import a custom Python class to help us manage media graphs. After the import, we will instantiate a graph manager object with IoT Hub, IoT Edge Device, and Graph API version details.

In [ ]:
import sys
sys.path.append('../../shared/graph_manager')
from graph_manager import GraphManager
from env_variables import *

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the deployment manifest file
operationsApiVersion = "1.0"  # Must be same as the version number in the deployment manifest file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

## List Existing Graph Topologies and Instances


In [ ]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

In [ ]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

## Set Topology
The following code snippet will deploy the "template.json" file into the lvaEdge module running on our IoT Edge device. The name of the topology file and other media graph related parameters must be set before in section named **"Setup the sample specific environment"**  

In [ ]:
# You can define a topologyFile or a topologyUrl that you want to deploy into the module. Here we point to our sample Media Graph Topology File.
import os.path

topologyFilePath =  os.path.join("../../../../MediaGraph/topologies/", topologyFile)

operationParams = {
                    "topologyFile": topologyFilePath
                    }

# Set Graph Topology
response = graphManager.GraphTopologySet(operationParams)
print(response)                    

## Set Topology Instance

The following code snippet will override and set the sample specific new values of the parameters mentioned in "template.json" and will create a Topology Instance.

In [ ]:
import json

with open(".media_graph_topology_parameters.json") as f:
    mediaGraphTopologyParameters = json.load(f)

# Set topology instance
response = graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)    

## Activate Topology Instance
Next, we activate the Topology Instance that we set in the previous node.

>[!IMPORTANT]  
>Be sure to set the name parameter below to the exact topology instance name that we used in the previous node.

In [ ]:
# Activate topology instance
operationParams = {
                    "name": graphInstanceName
                    }

response = graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

## Next Steps

If all the code cells above have successfully finished running, return to the Readme page to continue.   